# Running queries on the OMOP data

## Install required packages

Before executing the cells below, in a terminal session install the following required R packages:

`conda create -n omop-source r-glue r-tidyverse r-data.table r-dbi  r-rpostgres r-irkernel -y`

### Connect to the SQL database

In [ ]:
library(tidyverse)
library(data.table)
library(glue)
library(DBI)
library(RPostgres)

DBNAME <- #
HOST <- #
PORT <- #
PASSWORD <-  #
USER <- 'jupyter_notebook'

connection <- DBI::dbConnect(
    RPostgres::Postgres(),
    dbname = DBNAME,
    host = HOST,
    port = PORT,
    password = PASSWORD,
    user = USER,
    )

<br>  

## Running a basic select query

You can run queries using `dbGetQuery` and the `connection` created above. 

For example, to look at some fields from the OMOP `condition_occurrence` table:

In [ ]:
dbGetQuery(connection, "
    SELECT * FROM condition_occurrence LIMIT 5;
")

<br>

## Using the vocabulary tables

To make sense of records from the tables containing clinical data it is vital to understand the contents of `_concept_id` columns. This was discussed in the first training session but information can be found [here](https://ohdsi.github.io/TheBookOfOhdsi/StandardizedVocabularies.html#concepts).

### Searching the concept table

To find the appropriate concepts, you can either use [ATHENA](https://athena.ohdsi.org/search-terms/start) or query the vocabulary tables loaded in the database:


In [ ]:
laryngeal_cancer_concept <- dbGetQuery(connection, "
    SELECT * 
    FROM concept 
    WHERE concept_name ilike '%primary malignant neoplasm of larynx%'
    AND domain_id = 'Condition'
    AND standard_concept = 'S'
")

laryngeal_cancer_concept

### Using concept set to query event tables

In [ ]:
query <- "
SELECT count(distinct person_id)
FROM condition_occurrence
WHERE condition_concept_id in ({concept_list})
"
rendered_query <- glue(query, concept_list =  str_c(laryngeal_cancer_concept$concept_id, collapse = ", "))

dbGetQuery(connection, rendered_query)

### Taking advantage of heirarchical vocabularies

Find all patients with ear, nose, throat cancer without building a codelist by hand. Find the 'parent' concept:

In [ ]:
ent_cancer_concept <- dbGetQuery(connection, "
    SELECT * 
    FROM concept 
    WHERE concept_name ilike 'malignant tumor of ear, nose and throat'
    AND domain_id = 'Condition'
    AND standard_concept = 'S'
")

ent_cancer_concept

Then use `concept_ancestor` to find all 'children'

In [ ]:
query <- "
    SELECT count(distinct person_id)
    FROM condition_occurrence co
    INNER JOIN
        (SELECT descendant_concept_id
         FROM concept_ancestor
         WHERE ancestor_concept_id IN ({parent_concepts})) ca on ca.descendant_concept_id = co.condition_concept_id
"

rendered_query <- glue(query, parent_concepts =  str_c(ent_cancer_concept$concept_id, collapse = ", "))

dbGetQuery(connection, rendered_query)

<br>  

## More complicated queries 

### Example 1 (using one join)

Find all male patients with primary malignant neoplasm of larynx. Use the same query as we used to count all laryngeal cancer patients, but include a join to `person` where `gender_concept_id` corresponds to [males](https://athena.ohdsi.org/search-terms/terms/8507).

In [ ]:
query <- "
    SELECT count(distinct co.person_id)
    FROM condition_occurrence co
    INNER JOIN person p on co.person_id = p.person_id
    WHERE condition_concept_id IN ({concept_list})
    AND gender_concept_id = 8507
    LIMIT 5
"

rendered_query <- glue(
    query, 
    concept_list = str_c(laryngeal_cancer_concept$concept_id, collapse = ", ")
    )

dbGetQuery(connection, rendered_query)

### Example 2 (using two joins)

Find all male patients with primary malignant neoplasm of larynx was recorded in the [inpatient setting](https://athena.ohdsi.org/search-terms/terms/9201).

In [ ]:
query <- "
    SELECT count(distinct co.person_id)
    FROM condition_occurrence co
    INNER JOIN person p on co.person_id = p.person_id and gender_concept_id = 8507
    INNER JOIN visit_occurrence vo on co.visit_occurrence_id = vo.visit_occurrence_id and vo.visit_concept_id = 9201
    WHERE condition_concept_id IN ({concept_list})
    LIMIT 5
"

rendered_query <- glue(query, concept_list =  str_c(laryngeal_cancer_concept$concept_id, collapse = ", "))

dbGetQuery(connection, rendered_query)

### Example 3 (with inclusion and exclusion criteria)

Find all patients with ENT cancer **excluding** anyone who had laryngeal cancer.

In [ ]:
query <- "
WITH
    inclusion_criteria AS 
(
    SELECT distinct person_id
    FROM condition_occurrence co
    INNER JOIN (
        SELECT descendant_concept_id
        FROM concept_ancestor
        WHERE ancestor_concept_id IN ({concept_list_exclude})
    ) ca on ca.descendant_concept_id = co.condition_concept_id 
),

    exclusion_criteria AS
(
    SELECT distinct person_id
    FROM condition_occurrence co
    WHERE condition_concept_id IN ({concept_list_include})
)   
SELECT count(distinct person_id) FROM inclusion_criteria 
WHERE person_id not in (SELECT person_id FROM exclusion_criteria)
"

rendered_query <- glue(query, 
                       concept_list_include =  str_c(laryngeal_cancer_concept$concept_id, collapse = ", "),
                       concept_list_exclude =  str_c(ent_cancer_concept$concept_id, collapse = ", "))


dbGetQuery(connection, rendered_query)

### Example 4 (measurements and aggregate queries) 

Measurements are slightly different because they have a value as well as a concept_id. This is stored in either the `value_as_number`, `value_as_observation` or `value_as_string` columns.  

The below looks for all measurements matching the pattern `%height%` and returns their average value grouped by units.

In [ ]:
query <- "
    SELECT concept_name, vocabulary_id, concept_id, count(distinct person_id), unit_concept_id, unit_source_value, avg(value_as_number) 
    FROM measurement m
    INNER JOIN concept c on m.measurement_concept_id = c.concept_id
    WHERE concept_name ilike '%height%'
    GROUP BY concept_name,vocabulary_id, concept_id, unit_concept_id, unit_source_value
    ORDER BY count desc
"

dbGetQuery(connection, query)

### Example 5 (age of onset)

Calculate age of onset by taking the min condition start date and then subtracting year of birth.

**NOTE: there is a known bug in the data so year_of_birth is erroneous**

In [ ]:
query <- "
WITH CTE as (
    SELECT person_id, min(condition_start_date) onset_date
    FROM condition_occurrence
    WHERE condition_concept_id in ({concept_list})
    GROUP BY person_id
)
SELECT CTE.person_id, DATE_PART('year', onset_date) - p.year_of_birth as age_at_diagnosis
FROM CTE 
LEFT JOIN person_fix p on CTE.person_id = p.person_id
LIMIT 5
"
rendered_query <- glue(query, concept_list =  str_c(laryngeal_cancer_concept$concept_id, collapse = ", "))

dbGetQuery(connection, rendered_query)

### Example 6 (count number of diagnosis dates)

In [ ]:
query <- "
SELECT person_id, count(distinct condition_start_date) number_diagnoses
FROM condition_occurrence
WHERE condition_concept_id in ({concept_list})
GROUP BY person_id
ORDER BY number_diagnoses DESC
LIMIT 5
"
rendered_query <- glue(query, concept_list =  str_c(laryngeal_cancer_concept$concept_id, collapse = ", "))

dbGetQuery(connection, rendered_query)

<br>  

## Other ways to query

If you don't like using SQL but like using `dplyr`, the `dbplyr` table allows you to use `dbplyr` sytax to render queries.


In [ ]:
suppressPackageStartupMessages(library(dbplyr))

person <- tbl(connection, in_schema("omop_data_ukb_v1", "person"))
condition_occurrence <- tbl(connection, in_schema("omop_data_ukb_v1", "condition_occurrence"))

dbplyr_query <- condition_occurrence %>%
    filter(condition_concept_id == 26052) %>%
    inner_join(person, by = c("person_id" = "person_id")) %>%
    filter(gender_concept_id == 8507) %>%
    summarise(count = n_distinct(person_id))

dbplyr_query

In [ ]:
collect(dbplyr_query)